# Azure OpenAI Sora 2 quickstart (notebook)
Use this notebook to explore the Sora 2 video endpoints (create, poll status, download, list, delete) using the Azure OpenAI-compatible API surface.

## Setup
1. Create and activate a virtual environment.
2. Install the required packages (latest OpenAI client is required for `videos.*`):
```bash
python -m pip install --upgrade openai azure-identity python-dotenv
```
3. Copy `.env.example` to `.env` and fill in your Azure OpenAI values.
4. Restart the kernel after installing packages.

In [10]:
from __future__ import annotations

import os
import time
from pathlib import Path

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)

# Load environment values with safe defaults for the demo
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
SORA_MODEL = os.getenv("AZURE_OPENAI_SORA_DEPLOYMENT", "sora-2")
DEFAULT_SIZE = os.getenv("SORA_DEFAULT_SIZE", "720x1280")
DEFAULT_SECONDS = os.getenv("SORA_DEFAULT_SECONDS", "8")

if not ENDPOINT:
    raise ValueError("AZURE_OPENAI_ENDPOINT is required. Populate your .env file before running.")

print(f"Using endpoint: {ENDPOINT}")

def build_client() -> OpenAI:
    """Create an OpenAI client compatible with Azure OpenAI."""
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    if api_key:
        token_or_key = api_key
    else:
        # Fallback to Microsoft Entra ID (DefaultAzureCredential)
        token_or_key = get_bearer_token_provider(
            DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
        )

    return OpenAI(
        base_url=f"{ENDPOINT.rstrip('/')}/openai/v1/",
        api_key=token_or_key,
    )

client = build_client()
print("Client ready. Using deployment:", SORA_MODEL)

Using endpoint: https://aoai-hjfpii5xpvku4.cognitiveservices.azure.com/
Client ready. Using deployment: sora-2


In [11]:
# Create a video render job
prompt = "A cinematic drone shot of snow-capped mountains at sunrise, with snow covered dinosaur roaming majestically in the valley below."

video = client.videos.create(
    model=SORA_MODEL,
    prompt=prompt,
    size=DEFAULT_SIZE,
    seconds=DEFAULT_SECONDS,
)

VIDEO_ID = video.id
video

Video(id='video_6967be1b2d3881909f6792560c1ce698', completed_at=None, created_at=1768406555, error=None, expires_at=None, model='sora-2', object='video', progress=0, prompt='A cinematic drone shot of snow-capped mountains at sunrise, with snow covered dinosaur roaming majestically in the valley below.', remixed_from_video_id=None, seconds='8', size='720x1280', status='queued')

In [12]:
# Poll job status until completion (queued -> in_progress -> completed/failed)
POLL_INTERVAL_SECONDS = 20
status = video

while status.status not in {"completed", "failed", "cancelled"}:
    print(f"Status: {status.status} (progress: {getattr(status, 'progress', None)})")
    time.sleep(POLL_INTERVAL_SECONDS)
    status = client.videos.retrieve(status.id)

status

Status: queued (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)


Video(id='video_6967be1b2d3881909f6792560c1ce698', completed_at=1768406849, created_at=1768406555, error=None, expires_at=1768492955, model='sora-2', object='video', progress=100, prompt='A cinematic drone shot of snow-capped mountains at sunrise, with snow covered dinosaur roaming majestically in the valley below.', remixed_from_video_id=None, seconds='8', size='720x1280', status='completed')

In [13]:
# Download the completed video as an MP4
if status.status == "completed":
    output_dir = Path("outputs")
    output_dir.mkdir(exist_ok=True)
    video_path = output_dir / "sora_demo.mp4"

    content = client.videos.download_content(status.id, variant="video")
    content.write_to_file(str(video_path))
    print(f"Saved video to {video_path}")
else:
    raise RuntimeError(f"Video not completed. Current status: {status.status}")

Saved video to outputs\sora_demo.mp4


In [23]:
# List recent videos (paginated)
videos = client.videos.list(limit=5)
videos

SyncConversationCursorPage[Video](data=[Video(id='video_6966c008483c81908cec09773107192f', completed_at=1768341776, created_at=1768341512, error=None, expires_at=1768427912, model='sora-2', object='video', progress=100, prompt=None, remixed_from_video_id=None, seconds='4', size='720x1280', status='completed')], has_more=False, last_id='video_6966c008483c81908cec09773107192f', object='list', first_id='video_6966c008483c81908cec09773107192f')

In [24]:
# Delete a video by ID (replace if you want to clean up a different video)
video_id_to_delete = VIDEO_ID
delete_result = client.videos.delete(video_id_to_delete)
delete_result

VideoDeleteResponse(id='video_6966c008483c81908cec09773107192f', deleted=True, object='video.deleted')

In [14]:

# Create a video using a local reference image (must match requested size)
reference_path = Path("input_reference.png")
if not reference_path.exists():
    raise FileNotFoundError(f"Reference image not found: {reference_path}")

with reference_path.open("rb") as fh:
    reference_video = client.videos.create(
        model=SORA_MODEL,
        prompt="A video of a bird dancingg, bobbing their head to techno music",
        size=DEFAULT_SIZE,
        seconds=DEFAULT_SECONDS,
        input_reference=fh,
    )

REFERENCE_VIDEO_ID = reference_video.id
reference_video

Video(id='video_6967c12276708190999dad311c3ac42b', completed_at=None, created_at=1768407331, error=None, expires_at=None, model='sora-2', object='video', progress=0, prompt='A video of a bird dancingg, bobbing their head to techno music', remixed_from_video_id=None, seconds='8', size='720x1280', status='queued')

In [15]:
# Poll job status until completion (queued -> in_progress -> completed/failed)
POLL_INTERVAL_SECONDS = 20
status = reference_video

while status.status not in {"completed", "failed", "cancelled"}:
    print(f"Status: {status.status} (progress: {getattr(status, 'progress', None)})")
    time.sleep(POLL_INTERVAL_SECONDS)
    status = client.videos.retrieve(status.id)

status

Status: queued (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)


Video(id='video_6967c12276708190999dad311c3ac42b', completed_at=1768407635, created_at=1768407331, error=None, expires_at=1768493731, model='sora-2', object='video', progress=100, prompt='A video of a bird dancingg, bobbing their head to techno music', remixed_from_video_id=None, seconds='8', size='720x1280', status='completed')

In [16]:
# Download the completed video as an MP4
if status.status == "completed":
    output_dir = Path("outputs")
    output_dir.mkdir(exist_ok=True)
    video_path = output_dir / "sora_demo_input_reference.mp4"

    content = client.videos.download_content(status.id, variant="video")
    content.write_to_file(str(video_path))
    print(f"Saved video to {video_path}")
else:
    raise RuntimeError(f"Video not completed. Current status: {status.status}")

Saved video to outputs\sora_demo_input_reference.mp4


### Reference and remix variations
- **Reference image**: pass `input_reference=open("image.png", "rb")` in `videos.create` while matching the requested `size`.
- **URL reference**: fetch the image into a `BytesIO` object and pass it as `input_reference`.
- **Remix**: call `client.videos.remix(video_id=<completed_id>, prompt="Describe your change")` to adjust an existing asset while keeping structure/motion.

In [17]:
remix_video = client.videos.remix(video_id=status.id, prompt="Make the bird dance")

In [18]:
# Poll job status until completion (queued -> in_progress -> completed/failed)
POLL_INTERVAL_SECONDS = 20
status = remix_video

while status.status not in {"completed", "failed", "cancelled"}:
    print(f"Status: {status.status} (progress: {getattr(status, 'progress', None)})")
    time.sleep(POLL_INTERVAL_SECONDS)
    status = client.videos.retrieve(status.id)

status

Status: queued (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)
Status: in_progress (progress: 0)


Video(id='video_6967196cf64881908c5b352f258bcce8', completed_at=1768364562, created_at=1768364397, error=None, expires_at=1768450796, model='sora-2', object='video', progress=100, prompt='Make the bird dance', remixed_from_video_id='video_696717730078819082b51883851ecd3f', seconds='4', size='720x1280', status='completed')

In [19]:
# Download the completed video as an MP4
if status.status == "completed":
    output_dir = Path("outputs")
    output_dir.mkdir(exist_ok=True)
    video_path = output_dir / "sora_demo_remix.mp4"

    content = client.videos.download_content(status.id, variant="video")
    content.write_to_file(str(video_path))
    print(f"Saved video to {video_path}")
else:
    raise RuntimeError(f"Video not completed. Current status: {status.status}")

Saved video to outputs\sora_demo_remix.mp4
